# Building a Memory Graph

In this notebook, we'll explore how to build a memory graph to enhance GraphRAG with conversational context. We'll cover:

1. Memory graph structure
2. Tracking conversations and decisions
3. Using memory for better recommendations
4. Building a complete conversational agent

In [ ]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os
import openai
from datetime import datetime
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embedder import OpenAIEmbedder
from neo4j_graphrag.retriever import VectorRetriever, VectorCypherRetriever
from neo4j_graphrag.text2cypher import Text2Cypher

# Setup
load_dotenv()
driver = GraphDatabase.driver(
    os.getenv('NEO4J_URI'),
    auth=(os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD'))
)

openai.api_key = os.getenv('OPENAI_API_KEY')
llm = OpenAILLM()
embedder = OpenAIEmbedder()

## 1. Memory Graph Structure

Let's create a memory graph schema that tracks:

In [ ]:
def setup_memory_graph():
    with driver.session() as session:
        # Create constraints
        session.run("""
        CREATE CONSTRAINT memory_id IF NOT EXISTS
        FOR (m:Memory) REQUIRE m.id IS UNIQUE
        """)
        
        session.run("""
        CREATE CONSTRAINT conversation_id IF NOT EXISTS
        FOR (c:Conversation) REQUIRE c.id IS UNIQUE
        """)
        
        # Create indexes
        session.run("""
        CREATE INDEX memory_embedding IF NOT EXISTS
        FOR (m:Memory)
        ON (m.embedding)
        """)

setup_memory_graph()

## 2. Tracking Conversations and Decisions

Create a system to record interactions:

In [ ]:
class ConversationTracker:
    def __init__(self, driver, llm, embedder):
        self.driver = driver
        self.llm = llm
        self.embedder = embedder
        
    def start_conversation(self):
        with self.driver.session() as session:
            result = session.run("""
            CREATE (c:Conversation {
                id: randomUUID(),
                started: datetime(),
                active: true
            })
            RETURN c.id as conversation_id
            """)
            return result.single()["conversation_id"]
    
    def record_interaction(self, conversation_id, user_input, system_response, context_used=None):
        with self.driver.session() as session:
            # Create memory embedding
            memory_text = f"User: {user_input}\nSystem: {system_response}"
            embedding = self.embedder.embed_text(memory_text)
            
            # Record in graph
            session.run("""
            MATCH (c:Conversation {id: $conversation_id})
            
            CREATE (m:Memory {
                id: randomUUID(),
                timestamp: datetime(),
                user_input: $user_input,
                system_response: $system_response,
                embedding: $embedding,
                context_used: $context_used
            })
            
            CREATE (c)-[:CONTAINS]->(m)
            
            WITH m
            MATCH (prev:Memory)
            WHERE prev.timestamp < m.timestamp
            WITH m, prev
            ORDER BY prev.timestamp DESC
            LIMIT 1
            CREATE (prev)-[:NEXT]->(m)
            """, conversation_id=conversation_id,
                 user_input=user_input,
                 system_response=system_response,
                 embedding=embedding,
                 context_used=context_used)

# Create tracker
tracker = ConversationTracker(driver, llm, embedder)

# Example usage
conversation_id = tracker.start_conversation()
tracker.record_interaction(
    conversation_id,
    "What laptops do you recommend for video editing?",
    "Based on your needs, I recommend the XPS 17 with its powerful GPU and 4K display.",
    {"products_mentioned": ["XPS 17"], "features_considered": ["GPU", "4K display"]}
)

## 3. Using Memory for Better Recommendations

Enhance recommendations with conversation history:

In [ ]:
class MemoryAwareRecommender:
    def __init__(self, driver, llm, embedder):
        self.driver = driver
        self.llm = llm
        self.embedder = embedder
        self.retriever = VectorCypherRetriever(
            driver=driver,
            embedder=embedder,
            node_label="Memory",
            embedding_property="embedding"
        )
    
    def get_conversation_context(self, conversation_id):
        with self.driver.session() as session:
            result = session.run("""
            MATCH (c:Conversation {id: $conversation_id})-[:CONTAINS]->(m:Memory)
            RETURN m.user_input as input,
                   m.system_response as response,
                   m.context_used as context
            ORDER BY m.timestamp DESC
            LIMIT 5
            """, conversation_id=conversation_id)
            return list(result)
    
    def recommend(self, conversation_id, user_input):
        # Get conversation history
        history = self.get_conversation_context(conversation_id)
        
        # Find similar past interactions
        similar_memories = self.retriever.retrieve(user_input)
        
        # Build context-aware prompt
        prompt = f"""
        User Query: {user_input}
        
        Conversation History:
        {history}
        
        Similar Past Interactions:
        {similar_memories}
        
        Provide a recommendation that:
        1. Considers the user's current query
        2. Takes into account their conversation history
        3. Learns from similar past interactions
        4. Maintains consistency with previous responses
        """
        
        recommendation = self.llm.complete(prompt)
        return recommendation

# Create recommender
recommender = MemoryAwareRecommender(driver, llm, embedder)

# Example usage
response = recommender.recommend(
    conversation_id,
    "What accessories would work well with that laptop?"
)
print(response)

## 4. Building a Complete Conversational Agent

Put it all together into a GraphRAG agent:

In [ ]:
class GraphRAGAgent:
    def __init__(self, driver, llm, embedder):
        self.tracker = ConversationTracker(driver, llm, embedder)
        self.recommender = MemoryAwareRecommender(driver, llm, embedder)
        self.text2cypher = Text2Cypher(driver, llm)
    
    def chat(self, conversation_id, user_input):
        # Get recommendation
        response = self.recommender.recommend(conversation_id, user_input)
        
        # Record interaction
        self.tracker.record_interaction(conversation_id, user_input, response)
        
        return response

# Create agent
agent = GraphRAGAgent(driver, llm, embedder)

# Example conversation
conversation_id = agent.tracker.start_conversation()

queries = [
    "I need a laptop for video editing and 3D rendering",
    "What accessories would you recommend?",
    "Tell me more about the display options"
]

for query in queries:
    print(f"\nUser: {query}")
    response = agent.chat(conversation_id, query)
    print(f"Agent: {response}")

## Next Steps

You've now completed the GraphRAG workshop! Try:
1. Customizing the memory graph structure
2. Adding more sophisticated context tracking
3. Implementing your own GraphRAG patterns